# Calculating microbiome divergence rates

This notebook reimplements functions and ideas derived from [Nishida et al](https://onlinelibrary.wiley.com/doi/abs/10.1111/mec.14473), comparing evolutionary diversification among mammals to dissimilarities in their microbiomes. 

In [1]:
import numpy as np
import pandas as pd
import skbio as sk
from qiime2 import Artifact
from skbio import TreeNode
from plotnine import *
from os.path import join, abspath
from os import makedirs

from scipy.spatial.distance import squareform, pdist

## Calculate distance matrices

Calculate pairwise distance matrices (Qiime2?)
- Bray-Curtis
- Jaccard
- UniFrac, unweighted
- UniFrac, weighted

Load dissimilarity matrices

In [2]:
dm_dir = abspath('../bdiv')

dm_fps = {'Unweighted_UniFrac': 'unifrac_unw.merged-table.in-map.nomito-nochloro.10k.qza',
       'Weighted_UniFrac': 'unifrac_w.merged-table.in-map.nomito-nochloro.10k.qza',
       'Jaccard': 'jaccard.merged-table.in-map.nomito-nochloro.10k.qza'}

dms = {}

In [3]:
for metric in dm_fps:
    dm_art = Artifact.load(join(dm_dir, dm_fps[metric]))
    dms[metric] = dm_art.view(sk.DistanceMatrix)

load mapping file

In [5]:
md_dir = abspath('../metadata')
host_md_fp = '5per_10k.2.18.19.short.txt'
host_md = pd.read_csv(join(md_dir, host_md_fp), sep='\t')

In [6]:
host_md.shape

(2259, 67)

#### Subset tree and DMs to same tips

In [7]:
tree_dir = abspath('../trees/')
host_tree_fp = 'total_timetree_names.all.nwk.tre'

host_tree = sk.TreeNode.read(join(tree_dir, host_tree_fp), convert_underscores=False)

In [8]:
host_tips = [x.name for x in host_tree.tips()]

if subset, shear tree down to the subset:

In [9]:
subset = False

if subset:
    host_subset = np.random.choice(host_tips, size=100, replace=False)
else:
    host_subset = host_tips

host_tree_subset = host_tree.shear(host_subset)

In [10]:
len([x.name for x in host_tree_subset.tips()])

1333

In [11]:
host_ids_subset = host_md.loc[host_md['TimeTree_returned'].isin(host_subset), 'SampleID']

In [12]:
len(host_ids_subset)

2151

In [13]:
one_per_sp = False

if one_per_sp:
    host_md = host_md.loc[(host_md['SampleID'].isin(host_ids_subset)) &
                          (host_md['SampleID'].isin(unw_dm_subset.ids)),].groupby('TimeTree_returned').first()
    host_md =  host_md.loc[(host_md['SampleID'].isin(host_ids_subset)) &
                          (host_md['SampleID'].isin(unw_dm_subset.ids)),].groupby('TimeTree_returned').first().reset_index()
    host_ids_subset = list(set(host_ids_subset)) & set(host_md['SampleID'])

Filter distance matrices:

In [14]:
for dm in dms:
    host_ids_subset_dm = list(set(host_ids_subset) & set(dms[dm].ids))
    dms[dm] = dms[dm].filter(host_ids_subset_dm)

## Parse distance matrices

Subsample (group; pick arbitrary sample)

Convert to paired diagonal

Merge distance types

Add diet distances

Add phylo distances

Add taxonomic labels (S1, S2)

Add 'within' and 'between' taxonomic comparison columns

#### Subsample (group; pick arbitrary sample)

We don't want to confound this with within-species differences; subset matrix to one individual per species.

#### Convert to paired diagonal

We want a reduced form of the distance matrix that we can use for plotting.

#### Add diet distances

Let's get diet distances from our metadata

In [15]:
elton_cols = ['ET.Diet.Fruit',
             'ET.Diet.Inv',
             'ET.Diet.Nect',
             'ET.Diet.PlantO',
             'ET.Diet.Scav',
             'ET.Diet.Seed',
             'ET.Diet.Vect',
             'ET.Diet.Vend',
             'ET.Diet.Vfish',
             'ET.Diet.Vunk']

In [16]:
host_diet_df = host_md.loc[host_md[elton_cols].sum(axis=1) == 100,
                           ['SampleID'] + elton_cols].dropna()

In [17]:
host_diet_df.set_index('SampleID', inplace=True)

In [18]:
host_diet_df.head()

,ET.Diet.Fruit,ET.Diet.Inv,ET.Diet.Nect,ET.Diet.PlantO,ET.Diet.Scav,ET.Diet.Seed,ET.Diet.Vect,ET.Diet.Vend,ET.Diet.Vfish,ET.Diet.Vunk
SampleID,,,,,,,,,,
10684.weampp14D11,0.0,50.0,0.0,10.0,0.0,0.0,20.0,0.0,20.0,0.0
10684.weampp11B10,0.0,50.0,0.0,10.0,0.0,0.0,20.0,0.0,20.0,0.0
10684.weampp09H11,0.0,50.0,0.0,10.0,0.0,0.0,20.0,0.0,20.0,0.0
10684.weampp10A08,0.0,50.0,0.0,10.0,0.0,0.0,20.0,0.0,20.0,0.0
10684.weampp06A02,0.0,50.0,0.0,10.0,0.0,0.0,20.0,0.0,20.0,0.0


In [19]:
diet_dm = sk.DistanceMatrix(squareform(pdist(host_diet_df.iloc[:, :], metric='braycurtis')))
diet_dm.ids = host_diet_df.index

In [20]:
diet_dm.shape

(2053, 2053)

In [21]:
diet_art = Artifact.import_data('DistanceMatrix', diet_dm)

In [22]:
!ls ../

adiv
bdiv
bdtt
bdtt_files.tar.gz
bugbase
diet_distances.qza
eco_md-qiime_host_species_eco_metadata_by_SampleID_gut_11.28.18.txt
metadata
notebooks
pcoa_figs
phylosymbiosis
picrust
qiime2-2019.1-py36-linux-conda.yml
specificity_barcharts
specificity_stats
tables
taxonomy
testdm.qza
test_flight_group.qzv
trees


In [23]:
diet_art.save('../diet_distances.qza')

'../diet_distances.qza'

In [24]:
!readlink -f ../diet_distances.qza

/home/jgsanders/projects/templeton/201811/diet_distances.qza


### Add taxonomic info

In [25]:
host_lookup = {x['SampleID']: x['TimeTree_returned'] for i, x in host_md.iterrows()}

In [26]:
diet_dm_paired = diet_dm.to_series().reset_index()
diet_dm_paired.columns = ['host_1_id','host_2_id','diet_bc']

In [27]:
diet_dm_paired.head()

,host_1_id,host_2_id,diet_bc
0,10684.weampp14D11,10684.weampp11B10,0.0
1,10684.weampp14D11,10684.weampp09H11,0.0
2,10684.weampp14D11,10684.weampp10A08,0.0
3,10684.weampp14D11,10684.weampp06A02,0.0
4,10684.weampp14D11,10684.weampp05C04,0.3


In [28]:
host_lookup = {x['SampleID']: x['TimeTree_returned'] for i, x in host_md.iterrows()}

diet_dm_paired['host_1'] = diet_dm_paired['host_1_id'].apply(lambda x: host_lookup[x])
diet_dm_paired['host_2'] = diet_dm_paired['host_2_id'].apply(lambda x: host_lookup[x])

In [29]:
diet_dm_paired.head()

,host_1_id,host_2_id,diet_bc,host_1,host_2
0,10684.weampp14D11,10684.weampp11B10,0.0,Actitis_hypoleucos,Actitis_hypoleucos
1,10684.weampp14D11,10684.weampp09H11,0.0,Actitis_hypoleucos,Actitis_hypoleucos
2,10684.weampp14D11,10684.weampp10A08,0.0,Actitis_hypoleucos,Actitis_hypoleucos
3,10684.weampp14D11,10684.weampp06A02,0.0,Actitis_hypoleucos,Actitis_hypoleucos
4,10684.weampp14D11,10684.weampp05C04,0.3,Actitis_hypoleucos,Actitis_macularia


In [30]:
diet_dm_paired.shape

(2106378, 5)

In [31]:
distances = diet_dm_paired.copy()

#### Add taxonomic labels for each host


In [32]:
host_lookup_Class = {x['SampleID']: x['Taxonomy_Class'] for i, x in host_md.iterrows()}

distances['host_1_Class'] = distances['host_1_id'].apply(lambda x: host_lookup_Class[x])
distances['host_2_Class'] = distances['host_2_id'].apply(lambda x: host_lookup_Class[x])

In [33]:
host_lookup_Order = {x['SampleID']: x['Taxonomy_Order'] for i, x in host_md.iterrows()}

distances['host_1_Order'] = distances['host_1_id'].apply(lambda x: host_lookup_Order[x])
distances['host_2_Order'] = distances['host_2_id'].apply(lambda x: host_lookup_Order[x])

In [34]:
host_lookup_Family = {x['SampleID']: x['Taxonomy_Family'] for i, x in host_md.iterrows()}

distances['host_1_Family'] = distances['host_1_id'].apply(lambda x: host_lookup_Family[x])
distances['host_2_Family'] = distances['host_2_id'].apply(lambda x: host_lookup_Family[x])

In [35]:
distances.head()

,host_1_id,host_2_id,diet_bc,host_1,host_2,host_1_Class,host_2_Class,host_1_Order,host_2_Order,host_1_Family,host_2_Family
0,10684.weampp14D11,10684.weampp11B10,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae
1,10684.weampp14D11,10684.weampp09H11,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae
2,10684.weampp14D11,10684.weampp10A08,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae
3,10684.weampp14D11,10684.weampp06A02,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae
4,10684.weampp14D11,10684.weampp05C04,0.3,Actitis_hypoleucos,Actitis_macularia,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae


#### Add within and between fields

For subset graphing

In [36]:
distances['within_Class'] = False
distances.loc[distances['host_1_Class'] == distances['host_2_Class'],
                                              'within_Class'] = True

distances['within_Order'] = False
distances.loc[distances['host_1_Order'] == distances['host_2_Order'],
                                              'within_Order'] = True


distances['within_Family'] = False
distances.loc[distances['host_1_Family'] == distances['host_2_Family'],
                                              'within_Family'] = True

In [37]:
distances.head()

,host_1_id,host_2_id,diet_bc,host_1,host_2,host_1_Class,host_2_Class,host_1_Order,host_2_Order,host_1_Family,host_2_Family,within_Class,within_Order,within_Family
0,10684.weampp14D11,10684.weampp11B10,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True
1,10684.weampp14D11,10684.weampp09H11,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True
2,10684.weampp14D11,10684.weampp10A08,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True
3,10684.weampp14D11,10684.weampp06A02,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True
4,10684.weampp14D11,10684.weampp05C04,0.3,Actitis_hypoleucos,Actitis_macularia,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True


#### Remove between_class entries

In [38]:
distances = distances.loc[distances['within_Class'],]

In [39]:
distances.shape

(1074728, 14)

#### Add patristic distances

We can get these directly from the skbio treenode object

In [40]:
patristic_dm = host_tree_subset.tip_tip_distances()

#### Merge distance types 

This should just be a giant pandas d

Add host_species column to initial dm

In [41]:
def add_col(df, dm, col1, col2, name):
    
    df[name] = np.nan
    
    for i, row in df.iterrows():
        if i % 100000 == 0:
            print(i)
        try:
            df.loc[i, name] = dm[row[col1], row[col2]]
        except:
            continue


def get_col(df, dm, col1, col2):
    
    values = []

    for i, row in df.iterrows():
        if i % 100000 == 0:
            print(i)
        try:
            values.append(dm[row[col1], row[col2]])
        except:
            values.append(np.nan)

    return(pd.Series(values, index=df.index))




In [42]:
distances['phylo'] = get_col(distances, patristic_dm, 'host_1', 'host_2')

0
100000
200000
300000
500000
1200000
1300000
1400000
1700000
1900000
2100000


In [43]:
distances.shape

(1074728, 15)

In [44]:
distances.head()

,host_1_id,host_2_id,diet_bc,host_1,host_2,host_1_Class,host_2_Class,host_1_Order,host_2_Order,host_1_Family,host_2_Family,within_Class,within_Order,within_Family,phylo
0,10684.weampp14D11,10684.weampp11B10,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,0.0000
1,10684.weampp14D11,10684.weampp09H11,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,0.0000
2,10684.weampp14D11,10684.weampp10A08,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,0.0000
3,10684.weampp14D11,10684.weampp06A02,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,0.0000
4,10684.weampp14D11,10684.weampp05C04,0.3,Actitis_hypoleucos,Actitis_macularia,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,26.9633


#### Add microbiome distances

In [45]:
for metric in dms:
    distances[metric] = get_col(distances, dms[metric], 'host_1_id', 'host_2_id')

0
100000
200000
300000
500000
1200000
1300000
1400000
1700000
1900000
2100000
0
100000
200000
300000
500000
1200000
1300000
1400000
1700000
1900000
2100000
0
100000
200000
300000
500000
1200000
1300000
1400000
1700000
1900000
2100000


In [46]:
distances.head()

,host_1_id,host_2_id,diet_bc,host_1,host_2,host_1_Class,host_2_Class,host_1_Order,host_2_Order,host_1_Family,host_2_Family,within_Class,within_Order,within_Family,phylo,Unweighted_UniFrac,Weighted_UniFrac,Jaccard
0,10684.weampp14D11,10684.weampp11B10,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,0.0000,0.761690,0.445344,0.834711
1,10684.weampp14D11,10684.weampp09H11,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,0.0000,0.871487,0.578775,0.936232
2,10684.weampp14D11,10684.weampp10A08,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,0.0000,0.941777,0.730777,0.984890
3,10684.weampp14D11,10684.weampp06A02,0.0,Actitis_hypoleucos,Actitis_hypoleucos,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,0.0000,0.886211,0.905092,1.000000
4,10684.weampp14D11,10684.weampp05C04,0.3,Actitis_hypoleucos,Actitis_macularia,Aves,Aves,Charadriiformes,Charadriiformes,Scolopacidae,Scolopacidae,True,True,True,26.9633,0.810881,0.744436,1.000000


## Write total df to file

In [47]:
distances.loc[(distances['host_1_Order'] != distances['host_2_Order']) &
              (distances['host_1_Class'] == 'Mammalia'), 'Unweighted_UniFrac'].notna().sum()

291425

In [49]:
out_dir = '../phylosymbiosis'
makedirs(out_dir, exist_ok=True)

In [50]:
distances.to_csv(join(out_dir, 'phylo.distance_table.csv'))